## Xgboost train,testset에서 Class 1중 확률이 0.3~0.6인 sample extraction

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve, auc, log_loss
from sklearn.metrics import classification_report, precision_recall_curve, auc
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm

import torch 
import os
import torch

import warnings
warnings.filterwarnings("ignore")

In [2]:
is_cuda = torch.cuda.is_available()
device = torch.device('cuda' if is_cuda else 'cpu')
print(device)

cuda


In [3]:
## Load original data
data_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/Dataset/train_dataset.csv"
data = pd.read_csv(data_path, low_memory=False)

##load test data
test_data_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/Dataset/test_dataset.csv"
test_data = pd.read_csv(test_data_path)

In [6]:
data_x = data.drop(columns='loan_status')
data_y = data[['loan_status']]

X_test = test_data.drop(columns='loan_status')
y_test = test_data[['loan_status']]

In [7]:
# data under 6:2:2 train,val,test set
X_train, X_val, y_train, y_val = train_test_split(data_x, data_y, test_size=0.33, random_state=42, stratify=data_y)

print(f"Train size: {X_train.shape[0]} samples ({X_train.shape[0]/len(data):.1%})")
print(f"Validation size: {X_val.shape[0]} samples ({X_val.shape[0]/len(data):.1%})")
print(f"Test size: {X_test.shape[0]} samples ({X_test.shape[0]/len(data):.1%})")

Train size: 997369 samples (67.0%)
Validation size: 491242 samples (33.0%)
Test size: 372153 samples (25.0%)


In [8]:
train_dataset = pd.concat([X_train, y_train], axis=1)

fake_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/tablegan/samples/train/train_OI_11_00_fake.csv"
fake = pd.read_csv(fake_path)
fake['loan_status'] = 1

In [9]:
from sklearn.utils import shuffle

train_total = pd.concat([train_dataset,fake])
train_total = shuffle(train_total, random_state=42)

In [10]:
X_train = train_total.drop(columns='loan_status')
y_train = train_total[['loan_status']]

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier

#pos, neg = y_test.value_counts()  # ✅ `.values`로 NumPy 배열 변환
#scale_pos_weight = neg / pos  # 클래스 비율 계산

# 1️⃣ XGBoost 모델 정의
model = XGBClassifier(
    #scale_pos_weight=scale_pos_weight, 
    n_estimators=1000,
    learning_rate=0.01,
    objective='binary:logistic',
    eval_metric="logloss",
    random_state=42,
    verbose = True
)

In [ ]:
model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],  # Train & Validation Loss 저장
    verbose=True
)

In [22]:
# 5️⃣ 예측 및 평가
#y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]  # ROC Curve & PRC에 필요

threshold = 0.5
y_pred = (y_prob >= threshold).astype(int)

X_test_df = X_test.copy()
X_test_df['y_prob'] = y_prob
X_test_df['y_pred'] = y_pred
X_test_df['actual'] = y_test.values

filtered = X_test_df[(X_test_df['actual'] == 1)&
                     (X_test_df['y_prob']>=0.3)&
                     (X_test_df['y_prob']<=0.6)]

print(f"⚠️ 조건에 맞는 샘플 개수: {len(filtered)}")
display(filtered.head())  # Jupyter 환경이면 이렇게

⚠️ 조건에 맞는 샘플 개수: 7457


,last_fico_range_high,mo_sin_old_rev_tl_op,int_rate,annual_inc,dti,zip_code,installment,fico_range_low,total_rev_hi_lim,avg_cur_bal,bc_open_to_buy,revol_util,revol_bal,total_acc,credit_history_years,y_prob,y_pred,actual
20,579.0,112.0,0.1153,38000.0,26.34,846.0,277.12,660.0,9000.0,5176.0,1801.000000,0.635,5713.0,10.0,10,0.463981,0,1
31,584.0,39.0,0.1824,50300.0,20.07,24.0,590.25,720.0,5900.0,6294.0,11065.920936,0.201,1184.0,12.0,3,0.452710,0,1
107,559.0,122.0,0.1399,56000.0,27.93,483.0,239.21,665.0,3000.0,2336.0,0.000000,0.970,3008.0,31.0,11,0.502033,1,1
168,664.0,183.0,0.1311,76000.0,25.69,113.0,547.43,735.0,125600.0,7591.0,33170.000000,0.602,75614.0,19.0,16,0.435260,0,1
212,629.0,249.0,0.1333,245000.0,13.59,891.0,802.29,680.0,57100.0,40926.0,1976.000000,0.806,46024.0,28.0,21,0.401548,0,1


In [18]:
test_data['loan_status'].value_counts()

loan_status
0    299557
1     72596
Name: count, dtype: int64

In [28]:
filtered.head()

,last_fico_range_high,mo_sin_old_rev_tl_op,int_rate,annual_inc,dti,zip_code,installment,fico_range_low,total_rev_hi_lim,avg_cur_bal,bc_open_to_buy,revol_util,revol_bal,total_acc,credit_history_years,y_prob,y_pred,actual
20,579.0,112.0,0.1153,38000.0,26.34,846.0,277.12,660.0,9000.0,5176.0,1801.000000,0.635,5713.0,10.0,10,0.463981,0,1
31,584.0,39.0,0.1824,50300.0,20.07,24.0,590.25,720.0,5900.0,6294.0,11065.920936,0.201,1184.0,12.0,3,0.452710,0,1
107,559.0,122.0,0.1399,56000.0,27.93,483.0,239.21,665.0,3000.0,2336.0,0.000000,0.970,3008.0,31.0,11,0.502033,1,1
168,664.0,183.0,0.1311,76000.0,25.69,113.0,547.43,735.0,125600.0,7591.0,33170.000000,0.602,75614.0,19.0,16,0.435260,0,1
212,629.0,249.0,0.1333,245000.0,13.59,891.0,802.29,680.0,57100.0,40926.0,1976.000000,0.806,46024.0,28.0,21,0.401548,0,1


In [37]:
drop_columns = ['y_prob', 'y_pred', 'actual']
filtered = filtered.drop(columns=drop_columns)

In [38]:
filtered['loan_status'] = 1

In [39]:
path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/Dataset/"
filtered.to_csv(path_or_buf=path+"filtered_class1_0.3to0.6.csv", index=False)